<a href="https://colab.research.google.com/github/1900690/grape-count/blob/main/realtime-count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ウェブカメラに写ったものを数える
このノートでは、ウェブカメラで撮影した動画内の検出対象の数をカウントします。


In [8]:
#@title 事前準備
#yoloをインストールしてインポート
%pip -q install ultralytics
import ultralytics

#ライブラリをインポート
from ultralytics import YOLO
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

##関数定義
# 関数を使用して，JavaScript オブジェクトを OpenCV 画像に変換します。
def js_to_image(js_reply):
  """
  パラメータ
          js_reply： ウェブカメラからの画像を含む JavaScript オブジェクト
  戻り値
          img： OpenCV BGR 画像
  """
  # base64 imageをデコードする
  image_bytes = b64decode(js_reply.split(',')[1])
  # 配列をnumpy配列に変換する
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # numpy配列をOpenCV BGR画像にデコードする
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

#  OpenCVの矩形バウンディングボックス画像をBase64バイト文字列に変換してビデオストリームにオーバーレイする関数
def bbox_to_bytes(bbox_array):
  """
  パラメータ
          bbox_array： ビデオストリームにオーバーレイする矩形を含む Numpy 配列 (ピクセル)。
  戻り値
        バイト数： Base64 画像バイト列
  """
  # 配列をPIL画像に変換
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # bboxをpngに整形して返す
  bbox_PIL.save(iobuf, format='png')
  # フォーマットの戻り文字列
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes
# ウェブカメラを入力としてライブ・ビデオ・ストリームを適切に作成するためのJavaScript
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          '終了する場合は、ここかビデオをクリックしてください。</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.2/112.6 GB disk)


In [4]:
#検出するものに応じたPyTorch セーブファイルをロード　例:yolo11n.pt
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


In [16]:
#@title 検出
# ウェブカメラからのビデオストリーミング開始
video_stream()
# ビデオ用ラベル
label_html = '検出中…'
# バウンディングボックスを空に初期化する
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # JSレスポンスをOpenCV画像に変換
    img = js_to_image(js_reply["img"])

    # バウンディング・ボックスに透明オーバーレイを作成する
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    #検出
    results = model(img, verbose=False)

    #オーバーレイでボックスを取得
    for count,box in enumerate(results[0].boxes.xyxy.cpu().tolist(), start=1):
      x, y, w, h = map(int, box)
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    bbox_array=cv2.putText(bbox_array, str(count), (590, 50), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0,255,0), thickness=2)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    #  bboxのオーバーレイをバイトに変換
    bbox_bytes = bbox_to_bytes(bbox_array)
    #  次のフレームに新しいオーバーレイが表示されるように、bboxを更新する。
    bbox = bbox_bytes

<IPython.core.display.Javascript object>